In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout
import sklearn
from sklearn.model_selection import train_test_split
import os
from os import listdir
from zipfile import ZipFile
import pickle
import cv2
import gdown
import tensorflow
from tensorflow.keras.utils import img_to_array
import sklearn 
from sklearn.preprocessing import LabelBinarizer
import numpy as np


In [2]:
# %pip install opencv-python
# %pip install gdown

In [3]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            img = cv2.resize(image, (32,32))
            return img_to_array(img)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
def load_dataset(dir):
    ImgList, label_list = [], []
    try:
            print("Load Dataset -->")
            root_dir = listdir(dir)
            print("Root Directory : ", root_dir)
            for shape in root_dir:
                if shape == ".DS_Store":
                    root_dir.remove(dir)
            
            for shape in root_dir:
                faceShape_image_list = listdir(f"{dir}/{shape}")
                
                for faceShape in faceShape_image_list:
                    
                    if faceShape == ".DS_Store":
                        faceShape_image_lsit.remove(faceShape)
                        
                for faceShape_image in faceShape_image_list[:7000]:
                    
                    img_dir = f"{dir}/{shape}/{faceShape_image}"
                    
                    if img_dir.endswith(".jpg") == True or img_dir.endswith(".JPG") == True or img_dir.endswith(".png") == True or img_dir.endswith(".PNG") == True:
                        
                        converted_image = convert_image_to_array(img_dir)
                        ImgList.append(converted_image)
                        label_list.append(shape)
            print("Loading completed")
            return ImgList, label_list
    except Exception as e:
        print(f"Error : {e}")
        

In [5]:
images, labels = load_dataset("faceshape-master/published_dataset")

Load Dataset -->
Root Directory :  ['heart', 'oblong', 'oval', 'round', 'square']
Loading completed


In [8]:
print(len(images))
print(images[0].shape)

500
(32, 32, 3)


In [7]:
label_gen = LabelBinarizer()
image_labels = label_gen.fit_transform(labels)
pickle.dump(label_gen,open('label_transform.pkl', 'wb'))
n_classes = len(label_gen.classes_)
print(n_classes)
print(image_labels)

5
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [22]:
normalised_images = np.array(images, dtype=np.float16) /255.0

In [24]:
from sklearn.model_selection import train_test_split
print("making the training and testing split with 30% of the dataset as testing set")
# GAN_x_train, GAN_x_test, GAN_y_train, GAN_y_test = train_test_split(normalized_image_list_GAN, image_labels_GAN, test_size=0.3, shuffle=True,random_state = 42)

orignal_x_train, orignal_x_test, orignal_y_train, orignal_y_test = train_test_split(normalised_images, image_labels, test_size=0.3, shuffle=True,random_state = 42)
print(orignal_x_train.shape)

making the training and testing split with 30% of the dataset as testing set
(350, 0)


In [29]:
import tensorflow
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
# Define the input shape of the images
input_shape = (32, 32, 3)

# Initialize the model
model = Sequential()

# Add the first convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# Add the max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the second convolutional layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

# Add the max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add the flatten layer
model.add(Flatten())

# Add the dense layer
model.add(Dense(128, activation='relu'))

# Add the output layer
model.add(Dense(5, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the summary of the model architecture
model.summary()





Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)              

In [32]:
# Train the model
model.fit(orignal_x_train, orignal_y_train, epochs=10, batch_size=16)


Epoch 1/10


ValueError: in user code:

    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\jagmeet.singh\AppData\Lo

In [27]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

y_pred = model.predict(orignal_x_test)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)
# y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate the true labels for the test set
y_true = orignal_y_test

accuracy = accuracy_score(orignal_y_test, y_pred_binary)
precision = precision_score(orignal_y_test, y_pred_binary, average='macro')
recall = recall_score(orignal_y_test, y_pred_binary, average='macro')
f1 = f1_score(orignal_y_test, y_pred_binary, average='macro')
# conf_matrix = confusion_matrix(y_test, y_pred_binary)


print("Accuracy:", accuracy * 100)
print("Precision:", precision * 100)
print("Recall:", recall * 100)
print("F1-score:", f1 * 100)

ValueError: in user code:

    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "c:\Users\jagmeet.singh\AppData\Local\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\jagmeet.singh\AppDa